##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [ ]:
#%pip install pandas
#%pip install pyarrow
#%pip install numpy
#%pip install scikit-learn
#%pip install imbalanced-learn
#%pip install matplotlib

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [8]:
import pandas as pd

file_path = '/content/catB_train.parquet'

df = pd.read_parquet(file_path)

df.head()

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

In [ ]:
df.shape #to get the size of the dataframe

In [ ]:
df.info() #to understand the datatypes of dataframe

In [ ]:
df.isna().sum() #find out the number of missing values for each column

In [ ]:
df = df.drop_duplicates() #drop duplicate rows

To clean the dataframe, we will change all NAn values to 0 for f_purchase_lh and f_ever_declined_la, to indicate that clients will not purchase in the next 3 months. This is because we are basing it on the extreme and worse case scenario so that we will not overestimate our predictions for target column.

In [ ]:
selected_column_indices = list(range(1, 35)) + [-1]

# Create a new DataFrame with selected columns using iloc
df_clean = df.iloc[:, selected_column_indices]
df_clean.loc[:, "f_purchase_lh"] = df_clean.loc[:, "f_purchase_lh"].fillna(0)
df_clean.loc[:, "f_ever_declined_la"] = df_clean.loc[:, "f_ever_declined_la"].fillna(0)
df_clean.drop(columns=["hh_20","pop_20"],inplace=True)
df_clean.head()

In [ ]:
list(df_clean.columns.values) #this code is just for us to check which columns are left)

In [ ]:
column_data_types = df_clean.dtypes
print(column_data_types) #checking each columns' datatype

In [ ]:
import numpy as np
df_clean['client_risk'] = np.where(df_clean['flg_substandard'] == 1, 'Substandard',
                                np.where(df_clean['flg_is_borderline_standard'] == 1, 'Borderline Standard', 'Standard'))


# We combined flg_substandard, flg_is_borderline_standard into a new column client_risk, to label the types of each client

In [ ]:
df_clean.drop(columns=["flg_substandard","flg_is_borderline_standard"]) #remove the columns "flg_substandard" and "flg_is_borderline_standard"

In [ ]:
#code to help us understand size of each client type
import matplotlib.pyplot as plt
import seaborn as sns

client_risk_counts = df_clean['client_risk'].value_counts()

plt.figure(figsize=(10, 6))
sns.barplot(x=client_risk_counts.index, y=client_risk_counts.values, palette='viridis')
plt.title('Count of Each Client Risk Type')
plt.xlabel('Client Risk Type')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')

plt.show()

From here we understand that standard client risk types make up the largest group of customer base.

The code below is to create 3 piecharts to visualise the percentage of each type of clients who will purchase insurance plan in the next 3 months. This will let us know the behaviours of each client type and how our current outreach and interactions with them are going.

In [ ]:
client_risk_types = ['Substandard', 'Borderline Standard', 'Standard']

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Iterate over client risk types
for i, risk_type in enumerate(client_risk_types):
    # Filter the DataFrame to include only rows with the current client risk type
    df_risk_type = df_clean[df_clean['client_risk'] == risk_type]

    # Replace NaN values with 0 in the 'f_purchase_lh' column
    df_risk_type['f_purchase_lh'].fillna(0, inplace=True)

    # Calculate the percentage of people who made a purchase within the subset
    percentage_purchase = (df_risk_type['f_purchase_lh'].sum() / len(df_risk_type)) * 100

    # Create a pie chart for the current client risk type
    labels = ['Purchased', 'Not Purchased']
    sizes = [percentage_purchase, 100 - percentage_purchase]
    colors = ['#66b3ff', '#99ff99']  # Blue for Purchased, Green for Not Purchased

    axes[i].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    axes[i].set_title(f'Client Risk Type: {risk_type}')

# Set overall title
fig.suptitle('Percentage of Purchases Based on Client Risk Type')

# Adjust layout
plt.tight_layout()

# Show the subplots
plt.show()

In [ ]:
columns_of_interest = ['is_consent_to_mail', 'is_consent_to_email', 'is_consent_to_call',
                         'is_consent_to_sms']

# Melt the DataFrame to create a long-form representation
df_melted = pd.melt(df[columns_of_interest], var_name='Consent Type', value_name='Consent Value')

# Plot a side-by-side countplot
plt.figure(figsize=(15, 8))
sns.countplot(x='Consent Type', hue='Consent Value', data=df_melted, palette='Set2')

# Set plot labels and title
plt.xlabel('Consent Type')
plt.ylabel('Count')
plt.title('Counts of Consent Types (0 and 1)')

# Show the legend
plt.legend(title='Consent Value', loc='upper right', labels=['Not Granted', 'Granted'])

# Show the plot
plt.show()


This barchart shows an overview of the various types of communication that clients prefer. In this case, email is the most preferred form of communication. By extension, Singlife could choose to focus their marketing strategies and communication on insurance products to clients via email. On the other hand, the highest not granted form of communication is through call. Perhaps, Singlife can choose to remove call as a form as communication, since it may not be effective in reaching out to clients. This could also allow Singlife to achieve cost savings by reducing manpower in that aspect.

In [ ]:
communication_types = ['is_consent_to_mail', 'is_consent_to_email', 'is_consent_to_call', 'is_consent_to_sms']

# Create subplots
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Iterate over communication types
for i, comm_type in enumerate(communication_types):
    # Filter the DataFrame to include only rows where the current communication type is 1
    df_comm_type = df_clean[df_clean[comm_type] == 1]

    # Calculate the percentage of people who made a purchase within the subset
    percentage_purchase = (df_comm_type['f_purchase_lh'].sum() / len(df_comm_type)) * 100

    # Create a pie chart for the current communication type
    labels = ['Purchased', 'Not Purchased']
    sizes = [percentage_purchase, 100 - percentage_purchase]
    colors = ['#66b3ff', '#99ff99']  # Blue for Purchased, Green for Not Purchased

    axes[i].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    axes[i].set_title(f'Communication Type: {comm_type}')

# Set overall title
fig.suptitle('Percentage of People Who Agreed to Different Communication Types and Purchased')

# Adjust layout
plt.tight_layout()

# Show the subplots
plt.show()

The piecharts show the proportion of clients who will purchase health insurance products within 3 months for each mode of communication. Clients who gave consent to communication via email have the highest percentage of choosing to purchase the health insurance products. As seen, communication via email may be the most effective way to influence clients to purchase health insurance products. On the otherhand, communication via call witnessed the lowest percentage. Just like the previous graph, communication via call may not be the most optimal mode of communication for Singlife and clients.

In [ ]:
#making a copy of df_clean to remove all rows that contain at least one NAn value. This is because our values are either 0 or 1 (binary) so finding the mean/median for each value is not appropriate.
df_tree = df_clean.copy()
df_tree = df_tree.dropna()

The codes below is to make a decision tree to find out which of the 3 client risk type (standard, substandard and borderline standard) will be a key indicator of whether a client purchases within the next 3 months.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
import matplotlib.pyplot as plt

features = ['flg_has_health_claim', 'flg_has_life_claim', 'flg_gi_claim']
target = 'f_purchase_lh'
data = df_tree[features + [target]]

# Split the data into features (X) and target variable (y) to get training and test data
X = data[features]
y = data[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing a Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Fit the classifier on the training data
decision_tree_classifier.fit(X_train, y_train)

# Visualize the decision tree
plt.figure(figsize=(15, 10))
plot_tree(decision_tree_classifier, filled=True, feature_names=features, class_names=[str(x) for x in decision_tree_classifier.classes_])
plt.title("Decision Tree Visualization")
plt.show()

# Display the decision tree rules
tree_rules = export_text(decision_tree_classifier, feature_names=features)
print("Decision Tree Rules:\n", tree_rules)

# Evaluate the model on the test set (you might want to use more comprehensive evaluation metrics)
accuracy = decision_tree_classifier.score(X_test, y_test)
print(f"Accuracy on the test set: {accuracy:.2%}")


From the decision tree above, some key points that may result are:
Client having general claims have the largest role in predicting client NOT purchasing the policy, followed by health claims.

In [ ]:
import pandas as pd

# Get the data types of all columns in df_clean
dtypes = df_clean.dtypes

# Select only numeric columns
numeric_columns = dtypes[dtypes != "object"].index.tolist()

# Create a new DataFrame with only numeric columns
df_stepwise = df_clean[numeric_columns]
df_stepwise = df_stepwise.dropna()

# Print the shape of the new DataFrame
print("Shape of df_stepwise:", df_stepwise.shape)

Create new dataframe called df_stepwise for stepwise model selection to :choose a subset of predictor variables that optimally contribute to the predictive performance of the model. These new found predictors will be used to make a logistic regression model.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

df_stepwise = df_stepwise.dropna()
# Define the dependent and independent variables
y = df_stepwise["f_purchase_lh"]
X = df_stepwise.drop("f_purchase_lh", axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LogisticRegression model
model = LogisticRegression()

# Create the RFE object with the desired number of features
rfe = RFE(estimator=model, n_features_to_select=5)

# Fit the RFE object to the training data
rfe.fit(X_train, y_train)

# Get the mask of selected features
selected_features_mask = rfe.support_

# Get the names of selected features
selected_features = X_train.columns[selected_features_mask]

# Print the names of selected features
print("Selected features:", selected_features)

# Create a new X_train with only selected features
X_train_selected = X_train[selected_features]

# Train a LogisticRegression model with selected features
model.fit(X_train_selected, y_train)

# Evaluate the model on the test set
y_pred = model.predict(X_test[selected_features])
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy of the model
print("Accuracy:", accuracy)

From the above results, our selected features are: flg_gi_claim, flg_is_proposal, is_valid_dm, is_valid_email,  is_housewife_retiree.
We then make a logistic regression model.

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Define the dependent and independent variables
y = df_stepwise["f_purchase_lh"]
X = df_stepwise[['flg_gi_claim', 'flg_is_proposal', 'is_valid_dm', 'is_valid_email',
       'is_housewife_retiree']]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the logistic regression model
model = sm.Logit(y, X)

# Fit the model
results = model.fit()

# Print the results
print(results.summary())

This logistic regression model suggests that variables such as flg_gi_claim, flg_is_proposal, is_valid_dm, is_valid_email, and is_housewife_retiree are statistically significant predictors of the likelihood of the event f_purchase_lh, and thus critical touchpoints for SingLife.

The value of the log-likelihood function = 0.173166. In logistic regression, the likelihood function represents the probability of observing the given data under the estimated model. Lower values indicate a better fit.

Analysis:

From the above, we know critical touchpoint clients are those who are housewives or retired.

SingLife should also pay more attention to ensuring clients get their general insurance claims, as well as pushing to curate insurance proposal for clients. The best mode of communication for doing these would likely be email, as mentioned above.

In [ ]:
# Make predictions on the data
y_pred = results.predict(X)

# Convert the predictions to binary values
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]

# Calculate the accuracy of the model
accuracy = accuracy_score(y, y_pred_binary)

# Print the results
print("Accuracy:", accuracy)

## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [14]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    # Assuming df_tree is your DataFrame
    features = ['flg_has_health_claim', 'flg_has_life_claim', 'flg_gi_claim']
    target = 'f_purchase_lh'

    # Extract relevant features and target for the Decision Tree
    data = df_tree[features + [target]]

    # Split the data into features (X) and target variable (y)
    X = data[features]
    y = data[target]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a Decision Tree classifier
    decision_tree_classifier = DecisionTreeClassifier(random_state=42)
    decision_tree_classifier.fit(X_train, y_train)

    # Visualize the decision tree
    plt.figure(figsize=(15, 10))
    plot_tree(decision_tree_classifier, filled=True, feature_names=features, class_names=[str(x) for x in decision_tree_classifier.classes_])
    plt.title("Decision Tree Visualization")
    plt.show()

    # Display the decision tree rules
    tree_rules = export_text(decision_tree_classifier, feature_names=features)
    print("Decision Tree Rules:\n", tree_rules)

    # Predict missing values in the hidden data
    hidden_data_x = hidden_data[features]
    hidden_data_y_predicted = decision_tree_classifier.predict(hidden_data_x)

    return hidden_data_y_predicted.tolist()

# This cell should output a list of predicted binary classes.

hidden_data = pd.read_parquet(filepath)  # Load your hidden data
print(testing_hidden_data(hidden_data))

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!